<a href="https://colab.research.google.com/github/danyesss/NLP/blob/main/My_RuGPT3fine_tune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Finetune RuGPTs in huggingface
How to finetune RuGPTs models with huggingface. Example for RuGPT3Small. Nfor other models it will take more GPU memory.

This notebook is valid for all RuGPTs models except RuGPT3XL.
## Install env

In [ ]:
!rm -rf /usr/local/cuda
!ln -s /usr/local/cuda-10.1 /usr/local/cuda

In [1]:
%%bash
export LD_LIBRARY_PATH=/usr/lib/

In [2]:
!apt-get install clang-9 llvm-9 llvm-9-dev llvm-9-tools

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  binfmt-support libclang-common-9-dev libclang-cpp9 libclang1-9 libffi-dev
  libllvm9 libomp-9-dev libomp5-9 libpfm4 llvm-9-runtime python-chardet
  python-pkg-resources python-pygments python-yaml python3-pkg-resources
  python3-pygments python3-yaml
Suggested packages:
  clang-9-doc libomp-9-doc llvm-9-doc python-setuptools ttf-bitstream-vera
  python3-setuptools
The following NEW packages will be installed:
  binfmt-support clang-9 libclang-common-9-dev libclang-cpp9 libclang1-9
  libffi-dev libllvm9 libomp-9-dev libomp5-9 libpfm4 llvm-9 llvm-9-dev
  llvm-9-runtime llvm-9-tools python-chardet python-pkg-resources
  python-pygments python-yaml python3-pkg-resources python3-pygments
  python3-yaml
0 upgra

In [3]:
import subprocess

CUDA_version = [s for s in subprocess.check_output(["nvcc", "--version"]).decode("UTF-8").split(", ") if s.startswith("release")][0].split(" ")[-1]
print("CUDA version:", CUDA_version)

if CUDA_version == "10.0":
    torch_version_suffix = "+cu100"
elif CUDA_version == "10.1":
    torch_version_suffix = "+cu101"
elif CUDA_version == "10.2":
    torch_version_suffix = ""
else:
    torch_version_suffix = "+cu110"

CUDA version: 11.1


If code below doesn't work, check your cuda version and installation here https://pytorch.org/get-started/previous-versions/

In [4]:
!pip install torch==1.7.1{torch_version_suffix} torchvision==0.8.2{torch_version_suffix} torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |███████████████████████         | 834.1 MB 1.3 MB/s eta 0:04:11tcmalloc: large alloc 1147494400 bytes == 0x38e00000 @  0x7f7f9804b615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |█████████████████████████████▏  | 1055.7 MB 1.3 MB/s eta 0:01:18tcmalloc: large alloc 1434370048 bytes == 0x7d456000 @  0x7f7f9804b615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |█████

In [5]:
!pip install transformers==3.5.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 21.4 MB/s 
     |████████████████████████████████| 2.9 MB 50.8 MB/s 
     |████████████████████████████████| 880 kB 74.0 MB/s 
     |████████████████████████████████| 1.1 MB 69.2 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=4e49d333ad76b606feffc4cedf372ee67e2f3deb8a989ee2b344bb785dd34b49
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses


In [6]:
!git clone  https://github.com/sberbank-ai/ru-gpts

Cloning into 'ru-gpts'...
remote: Enumerating objects: 686, done.
remote: Counting objects: 100% (244/244), done.
remote: Compressing objects: 100% (79/79), done.
remote: Total 686 (delta 189), reused 165 (delta 165), pack-reused 442
Receiving objects: 100% (686/686), 377.77 KiB | 20.99 MiB/s, done.
Resolving deltas: 100% (428/428), done.


In [7]:
!mkdir models/

## Train 
The following code download our model and tokenizer from huggingface and finetune model for generating essays.

This took aroung ten minutes and obtain perplexity = 13-16

In [42]:
!export PYTHONPATH=${PYTHONPATH}:/ru-gpts/
!CUDA_VISIBLE_DEVICES=0 python ru-gpts/pretrain_transformers.py \
    --output_dir=models/poems \
    --model_type=gpt2 \
    --model_name_or_path=sberbank-ai/rugpt3small_based_on_gpt2 \
    --do_train \
    --train_data_file=brodsky_valid.txt \
    --do_eval \
    --eval_data_file=brodsky_valid.txt \
    --per_gpu_train_batch_size 1 \
    --gradient_accumulation_steps 1 \
    --num_train_epochs 20 \
    --block_size 1024 \
    --overwrite_output_dir    

08/03/2022 11:31:24 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:1423: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/modeling_auto.py:837: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,
08/03/2022 11:31:38 - INFO - __main__ -   Training/evaluation parameters Namespace(adam_epsilon=1e-08, block_size=1024, cache_dir=None, config_name=None, device=device(type='cuda'), do_eval=True, do_train=True, eval_all_checkpoints=False, eval_data_file='brodsky_va

## Evaluate model
### Generate with generate_transformers.py script.

### Generate mannualy

In [9]:
import numpy as np
import torch

In [10]:
np.random.seed(42)
torch.manual_seed(42)

In [11]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

In [43]:
tok = GPT2Tokenizer.from_pretrained("models/poems")

In [44]:
model = GPT2LMHeadModel.from_pretrained("models/poems")

In [45]:
model.cuda()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(2048, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): Laye

In [46]:
text = "Прощай,\n позабудь\n и не обессудь.\n А письма сожги,\n как мост."
inpt = tok.encode(text, return_tensors="pt")
out = model.generate(inpt.cuda(), max_length=200, repetition_penalty=5.0, do_sample=True, top_k=5, top_p=0.95, temperature=0.5)
tok.decode(out[0])

'Прощай,\n позабудь\n и не обессудь.\n А письма сожги,\n как мост. И в реку их бросить! Пусть ищут следов... Нощут всегда следы:). Ты все шумишь меж чужих досок;)), но услышишь лишь стук лопат по воде ( сквозь лопаты песка): « К нам идет Исаак!» — а под тобой вода с блестящим плеском бежит вдоль берега во тьму? Не видно лиц?.. Так высоко!.. Стук молотка внизу прекращает шум реки». Прислушайся к словам других людей. Если ты слышишь стукающий челн или лай бульдогов на склоне холма,— это значит только то же самое можно сказать о ком- нибудь из нас двоих., быть может даже об одном человеке.) Еще я помню одну песенку Авраама Шавкида (« Как он выглядит со стороны»). Песенка эта была ему перезвоном отговорив его от этой прогулки днем воскресенья следующего дня после заката солнца.— Пойдем туда вдвоем завтрашнего утра»,—'

In [18]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [40]:
!zip -r /content/model_3.zip /content/models/poems/checkpoint-4000 /content/models/poems/config.json /content/models/poems/eval_results.txt /content/models/poems/merges.txt /content/models/poems/pytorch_model.bin /content/models/poems/special_tokens_map.json /content/models/poems/tokenizer_config.json /content/models/poems/training_args.bin /content/models/poems/vocab.json

  adding: content/models/poems/checkpoint-4000/ (stored 0%)
  adding: content/models/poems/checkpoint-4000/config.json (deflated 48%)
  adding: content/models/poems/checkpoint-4000/pytorch_model.bin (deflated 16%)
  adding: content/models/poems/checkpoint-4000/vocab.json (deflated 76%)
  adding: content/models/poems/checkpoint-4000/training_args.bin (deflated 42%)
  adding: content/models/poems/checkpoint-4000/tokenizer_config.json (deflated 66%)
  adding: content/models/poems/checkpoint-4000/merges.txt (deflated 76%)
  adding: content/models/poems/checkpoint-4000/optimizer.pt (deflated 8%)
  adding: content/models/poems/checkpoint-4000/special_tokens_map.json (deflated 72%)
  adding: content/models/poems/checkpoint-4000/scheduler.pt (deflated 49%)
  adding: content/models/poems/config.json (deflated 48%)
  adding: content/models/poems/eval_results.txt (stored 0%)
  adding: content/models/poems/merges.txt (deflated 76%)
  adding: content/models/poems/pytorch_model.bin (deflated 16%)
  

In [41]:
!cp /content/model_3.zip /content/gdrive/MyDrive